In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split


In [3]:
# Dowload the dataset
dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
download_url(dataset_url, '.')

In [4]:
# Extract from archive
with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')

In [5]:
data_dir = './data/cifar10'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

In [6]:
airplane_files = os.listdir(data_dir + "/train/airplane")
print('No. of training examples for airplanes:', len(airplane_files))
print(airplane_files[:5])

In [7]:
ship_test_files = os.listdir(data_dir + "/test/ship")
print("No. of test examples for ship:", len(ship_test_files))
print(ship_test_files[:5])

In [8]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [9]:
dataset = ImageFolder(data_dir+'/train', transform=ToTensor())

In [10]:
img, label = dataset[40000]
print(img.shape, label)
img

In [11]:
print(dataset.classes)


In [12]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

matplotlib.rcParams['figure.facecolor'] = '#ffffff'

In [13]:
def show_example(img, label):
    print('Label: ', dataset.classes[label], "("+str(label)+")")
    plt.imshow(img.permute(1, 2, 0))

In [14]:
show_example(*dataset[0])


In [15]:
datasetTest = ImageFolder(data_dir+'/test', transform=ToTensor())

In [16]:
img, label = datasetTest[100]
print(img.shape, label)
img

In [17]:
#* actually spread the components for example - img, label = dataset[0]
show_example(*datasetTest[0])



In [18]:
show_example(*datasetTest[1000])

In [19]:
len(datasetTest)

In [20]:
import numpy as np

def split_indices(n, val_pct=0.1, seed=549):
    n_val = int(n * val_pct)
    np.random.seed(seed)
    idxs = np.random.permutation(n)
    return idxs[n_val:], idxs[:n_val]
    

In [21]:
val_pct = 0.2
rand_seed = 42

train_indices, val_indices = split_indices(len(dataset), val_pct, rand_seed)
print(len(train_indices), len(val_indices))

print('Sample Validation Indices: ', val_indices[:10])

In [22]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader

In [23]:
batch_size = 100

train_sampler = SubsetRandomSampler(train_indices)
train_dl = DataLoader(dataset,
                      batch_size,
                      sampler=train_sampler)


val_sampler = SubsetRandomSampler(val_indices)
val_dl = DataLoader(dataset,
                      batch_size,
                      sampler=val_sampler)

In [24]:
from torchvision.utils import make_grid

def show_batch(dl):
    for images, labels in dl:
        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(images, nrow=10).permute(1, 2, 0))
        break

In [25]:
show_batch(val_dl)

In [26]:
import torch.nn as nn
import torch.nn.functional as F

In [27]:
#to see the description of the nn.Conv2d fuction
?nn.Conv2d

In [28]:
simple_model = nn.Sequential(
    nn.Conv2d(3, 8, kernel_size=3, stride=1, padding=1),
    nn.MaxPool2d(2,2)
)

In [29]:
for images, labels in train_dl:
    print('images.shape: ', images.shape)
    # "out" is the feature map
    out = simple_model(images) 
    print('out.shape', out.shape)
    break

In [30]:
model = nn.Sequential(
    #PyTorch channels are shown before image dimmensions
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),         # it's written in this way:- batch_size x channels x image shape(row x col)
    nn.MaxPool2d(2,2), #output: batch_size x 16(ch) x 16 x 16 (half of input layer)
    
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2), #output: batch_size x 16(ch) x 8 x 8(half of previous layer)
    
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2), #output: batch_size x 16(ch) x 4 x 4 (half of input layer)
    
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2), #output: batch_size x 16(ch) x 2 x 2 (half of previous layer)
    
    nn.Conv2d(16, 16, kernel_size=3, stride=1, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2,2), #output: batch_size x 16(ch) x 1 x 1 (half of previous layer)
    
    nn.Flatten(), #output: batch_size x 16
    nn.Linear(16, 10) #output: batch_size x 10
)

In [31]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    

def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        for b in self.dl:
            yield to_device(b, self.device)
            
    def __len__(self):
        return len(self.dl)
            
    

In [32]:
device = get_default_device()
device

In [33]:
#if you're using CUDA then your weights and biases as well as your inputs should also
#be in CUDA
if torch.cuda.is_available():
    model.cuda()

In [34]:
train_dl = DeviceDataLoader(train_dl, device)
valid_dl = DeviceDataLoader(val_dl, device)

In [35]:
def loss_batch(model, loss_func, xb, yb, opt=None, metric=None):
    preds = model(xb)
    loss = loss_func(preds, yb) #cross_entropy
    
    #this is used in trainig not in validation
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()
        
    metric_result = None
    
    if metric is not None:
        metric_result = metric(preds, yb)
    
    return loss.item(), len(xb), metric_result
        

In [36]:
def evaluate(model, loss_fn, valid_dl, metric=None):
    with torch.no_grad():
        results = [loss_batch(model, loss_fn, xb, yb, metric=metric)
                  for xb, yb in valid_dl]
        losses, nums, metrics = zip(*results)
        total = np.sum(nums)
        avg_loss = np.sum(np.multiply(losses, nums)) / total
        avg_metric = None
        if metric is not None:
            avg_metric = np.sum(np.multiply(metrics, nums)) / total
    return avg_loss, total, avg_metric

In [37]:
def fit(epochs, model, loss_fn, train_dl, valid_dl,
        opt_fn=None, lr=None, metric=None):
    
    train_losses, val_losses, val_metrics = [], [], []

    if opt_fn is None:
        opt_fn = torch.optim.SGD
    # optimizer is instantiated with model's parameters and learning rate
    opt = opt_fn(model.parameters(), lr=lr)
    
    for epoch in range(epochs):
        # this is a pytorch setting that tells you whether you are training or evaluating
        model.train()
        for xb, yb in train_dl:
            train_loss,_,_ = loss_batch(model, loss_fn, xb, yb, opt)
            
        # you're telling pytorch that you're evaluating r8 now
        model.eval()
        result = evaluate(model, loss_fn, valid_dl, metric)
        val_loss, total, val_metric = result
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_metrics.append(val_metric)
        
        if metric is None:
            print('Epoch [{}/{}], train_loss: {:4f}, val_{}: {:4f})' 
                  .format(epoch+1, epochs, train_loss, val_loss,
                         metric))
        else:
            print('Epoch [{}/{}], train_loss: {:.4f}, val_loss: {:.4f}, val_{}: {:.4f}'
                 .format(epoch+1, epochs, train_loss, val_loss, 
                        metric.__name__, val_metric))
    return train_losses, val_losses, val_metrics
            

In [38]:
def accuracy(outputs, labels):
    _,preds = torch.max(outputs, dim=1)
    return torch.sum(preds == labels).item() / len(preds)
    

In [39]:
val_loss, _, val_acc = evaluate(model, F.cross_entropy, 
                                valid_dl, metric=accuracy)
print('Loss: {:.4f}, Accuracy: {:.4f}' .format(val_loss, val_acc))

In [40]:
#hyperparameters
num_epochs = 10
opt_fn = torch.optim.Adam
lr = 0.005

In [41]:
history = fit(num_epochs, model, F.cross_entropy, 
              train_dl, valid_dl, opt_fn, lr, accuracy)
train_losses, val_losses, val_metrics = history

In [42]:
def plot_metric(metric_values):
    plt.plot(metric_values, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy VS # of Epochs')

In [43]:
plot_metric([val_acc] + val_metrics)

In [44]:
def plot_losses(train_losses, val_losses):
    plt.plot(train_losses, '-x')
    plt.plot(val_losses, '-o')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss VS # of Epochs')

In [45]:
plot_losses([None]+train_losses, [val_loss]+ val_losses)